In [1]:
import numpy as np
from ikpy.chain import Chain
from ikpy.link import URDFLink
from scipy.spatial.transform import Rotation as R
from ikpy.utils import geometry
import os

joint_limits_min = np.array([-166, -101, -166, -176, -166, -1, -166])
joint_limits_max = np.array([166, 101, 166, -4, 166, 215, 166])
lims_min_safe = joint_limits_min + 1
lims_max_safe = joint_limits_max -1
joint_limits_rad_min = np.radians(lims_min_safe) 
joint_limits_rad_max = np.radians(lims_max_safe) 
prev_angle = np.random.uniform(joint_limits_rad_min, joint_limits_rad_max).tolist()

franka_chain = Chain.from_urdf_file(
    "franka.urdf",
    base_elements=["panda_panda_link0"],
    last_link_vector=np.array([0, 0, 0.045]),
    active_links_mask = [False, True, True, True, True, True, True, True, False, False, False, False]
)
#set bounds
for i in range(len(joint_limits_rad_min)):
    franka_chain.links[i + 1].bounds = (joint_limits_rad_min[i], joint_limits_rad_max[i])


validity_time = 0
setup_time = 0
solve_time = 0

In [2]:
#!/usr/bin/env python
"""Example demonstrating collision detection and shortest distance queries."""

import time
import numpy as np
import pybullet as pyb
import pybullet_data
import pyb_utils

TIMESTEP = 1.0 / 60
all_x, all_y, all_z = [],[],[]
def load_environment(client_id, difficulty):
    if difficulty == 1:
        pyb.setTimeStep(TIMESTEP, physicsClientId=client_id)
        pyb.setAdditionalSearchPath(
            pybullet_data.getDataPath(), physicsClientId=client_id
        )
        
        # ground plane
        ground_id = pyb.loadURDF(
            "plane.urdf", [0, 0, 0], useFixedBase=True, physicsClientId=client_id
        )
    
        # KUKA iiwa robot arm (franka)
        franka_id = pyb.loadURDF(
            "franka.urdf",
            [0, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        robot = pyb_utils.Robot(franka_id, client_id=client_id)
        for i in range(pyb.getNumJoints(franka_id)):
            joint_info = pyb.getJointInfo(franka_id, i)
            joint_name = joint_info[1].decode("utf-8")
            if joint_name == "panda_finger_joint1" or joint_name == "panda_finger_joint2":
                pyb.resetJointState(franka_id, i, 0.04, physicsClientId=client_id)
            
        # some cubes for obstacles
        cube1_id = pyb.loadURDF(
            "WallObstacle.urdf", useFixedBase=True, physicsClientId=client_id
        )
        targetcube = pyb.loadURDF(
            "TargetCube.urdf",
            [0, 0.5, 0.1],
            useFixedBase=True,
            physicsClientId=client_id,
        )
    
        obstacles = {
            "ground": ground_id,
            "cube1": cube1_id,
            "targetcube": targetcube
        }
        return robot, obstacles
    elif difficulty == 2:
        print(pybullet_data.getDataPath())
        pyb.setTimeStep(TIMESTEP, physicsClientId=client_id)
        pyb.setAdditionalSearchPath(
            pybullet_data.getDataPath(), physicsClientId=client_id
        )

        # ground plane
        ground_id = pyb.loadURDF(
            "plane.urdf", [0, 0, 0], useFixedBase=True, physicsClientId=client_id
        )

        # Roboter
        franka_id = pyb.loadURDF(
            "franka.urdf",
            [0, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        robot = pyb_utils.Robot(franka_id, client_id=client_id)
        for i in range(pyb.getNumJoints(franka_id)):
            joint_info = pyb.getJointInfo(franka_id, i)
            joint_name = joint_info[1].decode("utf-8")
            if joint_name in ("panda_finger_joint1", "panda_finger_joint2"):
                pyb.resetJointState(franka_id, i, 0.04, physicsClientId=client_id)

        # ---- Konvexe Dekomposition vorbereiten ----
        mesh_path = "meshes/SM_BucketPlastic_B.obj"  
        vhacd_output = "meshes/mesh_vhacd.obj"  
        # Führe VHACD nur einmal aus 
        if not os.path.exists(vhacd_output):
            pyb.vhacd(mesh_path, vhacd_output, "vhacd_log.txt")

        # Lade VHACD-Ergebnis als Hindernis (direkt als Visual+Collision Shape)
        visual_shape = pyb.createVisualShape(pyb.GEOM_MESH, fileName=vhacd_output, meshScale=[0.01, 0.01, 0.003], physicsClientId=client_id)
        collision_shape = pyb.createCollisionShape(pyb.GEOM_MESH, fileName=vhacd_output, meshScale=[0.01, 0.01, 0.003], physicsClientId=client_id)

        cube1_id = pyb.createMultiBody(
            baseMass=0,
            baseCollisionShapeIndex=collision_shape,
            baseVisualShapeIndex=visual_shape,
            basePosition=[0, 0.5, 0],
            useMaximalCoordinates=True,
            physicsClientId=client_id
        )

        

        targetcube = pyb.loadURDF(
            "TargetCube.urdf",
            [0, 0.5, 0.1],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        obstacles = {
            "ground": ground_id,
            "cube1": cube1_id,
            "targetcube": targetcube
        }
        return robot, obstacles    
    
    elif difficulty == 3:
        pyb.setTimeStep(TIMESTEP, physicsClientId=client_id)
        pyb.setAdditionalSearchPath(
            pybullet_data.getDataPath(), physicsClientId=client_id
        )

        # ground plane
        ground_id = pyb.loadURDF(
            "plane.urdf", [0, 0, 0], useFixedBase=True, physicsClientId=client_id
        )

        # Roboter
        franka_id = pyb.loadURDF(
            "franka.urdf",
            [0, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        robot = pyb_utils.Robot(franka_id, client_id=client_id)
        for i in range(pyb.getNumJoints(franka_id)):
            joint_info = pyb.getJointInfo(franka_id, i)
            joint_name = joint_info[1].decode("utf-8")
            if joint_name in ("panda_finger_joint1", "panda_finger_joint2"):
                pyb.resetJointState(franka_id, i, 0.04, physicsClientId=client_id)

        # ---- Konvexe Dekomposition vorbereiten ----
        mesh_path = "meshes/SM_BucketPlastic_B.obj"  
        vhacd_output = "meshes/mesh_vhacd.obj"  
        # Führe VHACD nur einmal aus 
        if not os.path.exists(vhacd_output):
            pyb.vhacd(mesh_path, vhacd_output, "vhacd_log.txt")

        # Lade VHACD-Ergebnis als Hindernis (direkt als Visual+Collision Shape)
        visual_shape = pyb.createVisualShape(pyb.GEOM_MESH, fileName=vhacd_output, meshScale=[0.01, 0.01, 0.005], physicsClientId=client_id)
        collision_shape = pyb.createCollisionShape(pyb.GEOM_MESH, fileName=vhacd_output, meshScale=[0.01, 0.01, 0.005], physicsClientId=client_id)

        cube1_id = pyb.createMultiBody(
            baseMass=0,
            baseCollisionShapeIndex=collision_shape,
            baseVisualShapeIndex=visual_shape,
            basePosition=[0, 0.5, 0],
            useMaximalCoordinates=True,
            physicsClientId=client_id
        )
        cube2_id = pyb.createMultiBody(
            baseMass=0,
            baseCollisionShapeIndex=collision_shape,
            baseVisualShapeIndex=visual_shape,
            basePosition=[0.5, 0, 0],
            useMaximalCoordinates=True,
            physicsClientId=client_id
        )
        targetcube = pyb.loadURDF(
            "TargetCube.urdf",
            [0, 0.5, 0.1],
            useFixedBase=True,
            physicsClientId=client_id,
        )

        obstacles = {
            "ground": ground_id,
            "cube1": cube1_id,
            "cube2": cube2_id,
            "targetcube": targetcube
        }
        return robot, obstacles  

    elif difficulty == 4:
        pyb.setTimeStep(TIMESTEP, physicsClientId=client_id)
        pyb.setAdditionalSearchPath(
            pybullet_data.getDataPath(), physicsClientId=client_id
        )

        # ground plane
        ground_id = pyb.loadURDF(
            "plane.urdf", [0, 0, 0], useFixedBase=True, physicsClientId=client_id
        )
        print(pybullet_data.getDataPath())
        # Roboter

        franka_id = pyb.loadURDF(
            "franka.urdf",
            [0, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        print(franka_id)
        robot = pyb_utils.Robot(franka_id, client_id=client_id)
        for i in range(pyb.getNumJoints(franka_id)):
            joint_info = pyb.getJointInfo(franka_id, i)
            joint_name = joint_info[1].decode("utf-8")
            if joint_name in ("panda_finger_joint1", "panda_finger_joint2"):
                pyb.resetJointState(franka_id, i, 0.04, physicsClientId=client_id)

        table = pyb.loadURDF(
            "Tablecube.urdf",
            [0, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        soup1 = pyb.loadURDF(
            "tomato_soup_can1.urdf",
            [0, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        soup2 = pyb.loadURDF(
            "tomato_soup_can2.urdf",
            [0, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        targetcube = pyb.loadURDF(
            "TargetCube.urdf",
            [0.69, 0, 0],
            useFixedBase=True,
            physicsClientId=client_id,
        )
        
       # ---- Konvexe Dekomposition vorbereiten ----
        mesh_path = "meshes/SM_BucketPlastic_B.obj"  
        vhacd_output = "meshes/mesh_vhacd.obj"  
        # Führe VHACD nur einmal aus 
        if not os.path.exists(vhacd_output):
            pyb.vhacd(mesh_path, vhacd_output, "vhacd_log.txt")

        # Lade VHACD-Ergebnis als Hindernis (direkt als Visual+Collision Shape)
        visual_shape = pyb.createVisualShape(pyb.GEOM_MESH, fileName=vhacd_output, meshScale=[0.01, 0.01, 0.005], physicsClientId=client_id)
        collision_shape = pyb.createCollisionShape(pyb.GEOM_MESH, fileName=vhacd_output, meshScale=[0.01, 0.01, 0.005], physicsClientId=client_id)

        cube1_id = pyb.createMultiBody(
            baseMass=0,
            baseCollisionShapeIndex=collision_shape,
            baseVisualShapeIndex=visual_shape,
            basePosition=[0.69, 0, 0],
            useMaximalCoordinates=True,
            physicsClientId=client_id
        )
        
        obstacles = {
            "ground": ground_id,
            "table": table,
            "soup1": soup1,
            "soup2": soup2,
            #"box": box,
            "targetcube": targetcube,
        }
        return robot, obstacles   
  

def setup_collision_env(env_num):
    col_id = pyb.connect(pyb.DIRECT)
    print(1)
    col_robot, col_obstacles = load_environment(col_id, env_num)
    links = [
        "panda_panda_link1", "panda_panda_link2", "panda_panda_link3",
        "panda_panda_link4", "panda_panda_link5", "panda_panda_link6",
        "panda_panda_link7", "panda_panda_hand",
        "panda_panda_leftfinger", "panda_panda_rightfinger"
    ]
    link_tuples = [(col_robot.uid, link_name) for link_name in links]
    obstacles = list(col_obstacles.values())
    collision_pairs = [(obs, link) for obs in obstacles for link in link_tuples]

    return col_id, col_robot, collision_pairs

def calculate_state_validity(current_joint_state, col_robot, collision_pairs, col_id, timers=None):
    if timers is None:
        timers = {"validity": 0.0, "collision": 0.0}

    run_start_time = time.time()

    if len(current_joint_state) == 7:
        full_angles = franka_chain.active_to_full(current_joint_state, np.zeros(len(franka_chain.links)))
    else:
        full_angles = current_joint_state

    start_collision_time = time.time()
    col_detector = pyb_utils.CollisionDetector(col_id, collision_pairs)
    if timers is not None:
        timers["collision"] += time.time() - start_collision_time

    col_robot.reset_joint_configuration(current_joint_state)

    if timers is not None:
        timers["validity"] += time.time() - run_start_time

    if not col_detector.in_collision(margin=0.01):
        #print(f"State valid")
        return True
    else: 
        #print(f"State invalid")
        return False

if __name__ == "__main__":
    current_state = [ 0.49749239, -0.39204911, -0.38585028, -1.32839171,  1.41997276,
  0.3847102,  -1.79493424]

    #valid = calculate_state_validity(current_state)
    


pybullet build time: Jun 10 2025 11:09:21


In [3]:
import sys
import time
try:
    from ompl import util as ou
    from ompl import base as ob
    from ompl import geometric as og
except ImportError:
    # if the ompl module is not in the PYTHONPATH assume it is installed in a
    # subdirectory of the parent directory called "py-bindings."
    from os.path import abspath, dirname, join
    sys.path.insert(0, join(dirname(dirname(abspath(__file__))), 'py-bindings'))
    from ompl import util as ou
    from ompl import base as ob
    from ompl import geometric as og
from math import sqrt
import argparse

j = 0
tested_joint_states = []
class ValidityChecker(ob.StateValidityChecker):
    def __init__(self, si, min_distance, timers, col_robot, collision_pairs, col_id):
        super(ValidityChecker, self).__init__(si)
        self.min_distance = min_distance
        self.timers = timers
        self.col_robot = col_robot
        self.collision_pairs = collision_pairs
        self.col_id = col_id
        
        
        
    # Returns whether the given state's position overlaps the
    # circular obstacle
    
    def isValid(self, state):       
        joint_angles = [state[i] for i in range(7)]
        global j
        j = j+1
        #full_angles = franka_chain.active_to_full(joint_angles, np.zeros(len(franka_chain.links)))
        # current_fk = franka_chain.forward_kinematics(full_angles)
        # x, y, z = current_fk[:3, 3]
        # all_x.append(x)
        # all_y.append(y)
        # all_z.append(z)
        valid = calculate_state_validity(joint_angles, self.col_robot, self.collision_pairs, self.col_id, timers=self.timers)
        tested_joint_states.append(joint_angles[:])
        if not valid:
            print(f"Invalid state: {joint_angles}")
        return valid
        #return True

    # Returns the distance from the given state's position to the
    # boundary of the circular obstacle.
    def clearance(self, state):
        # Extract the robot's (x,y) position from its state
        print("hallo")
        x = state[0]
        y = state[1]
        z = state[2]

  
        

        # Distance formula between three points, offset by the circle's
        # radius
        return sqrt((x - 0.5)**2 + (y - 0.5)**2 + (z - 0.5)**2) - 0.05

    def checkCollisions(self, state):
        pass

## Returns a structure representing the optimization objective to use
#  for optimal motion planning. This method returns an objective
#  which attempts to minimize the length in configuration space of
#  computed paths.
def getPathLengthObjective(si):
    return ob.PathLengthOptimizationObjective(si)

## Returns an optimization objective which attempts to minimize path
#  length that is satisfied when a path of length shorter than 1.51
#  is found.
def getThresholdPathLengthObj(si):
    obj = ob.PathLengthOptimizationObjective(si)
    obj.setCostThreshold(ob.Cost(1.51))
    return obj

## Defines an optimization objective which attempts to steer the
#  robot away from obstacles. To formulate this objective as a
#  minimization of path cost, we can define the cost of a path as a
#  summation of the costs of each of the states along the path, where
#  each state cost is a function of that state's clearance from
#  obstacles.
#
#  The class StateCostIntegralObjective represents objectives as
#  summations of state costs, just like we require. All we need to do
#  then is inherit from that base class and define our specific state
#  cost function by overriding the stateCost() method.
#
class ClearanceObjective(ob.StateCostIntegralObjective):
    def __init__(self, si):
        super(ClearanceObjective, self).__init__(si, True)
        self.si_ = si

    # Our requirement is to maximize path clearance from obstacles,
    # but we want to represent the objective as a path cost
    # minimization. Therefore, we set each state's cost to be the
    # reciprocal of its clearance, so that as state clearance
    # increases, the state cost decreases.
    def stateCost(self, s):
        return ob.Cost(1 / (self.si_.getStateValidityChecker().clearance(s) +
                            sys.float_info.min))

## Return an optimization objective which attempts to steer the robot
#  away from obstacles.
def getClearanceObjective(si):
    return ClearanceObjective(si)

## Create an optimization objective which attempts to optimize both
#  path length and clearance. We do this by defining our individual
#  objectives, then adding them to a MultiOptimizationObjective
#  object. This results in an optimization objective where path cost
#  is equivalent to adding up each of the individual objectives' path
#  costs.
#
#  When adding objectives, we can also optionally specify each
#  objective's weighting factor to signify how important it is in
#  optimal planning. If no weight is specified, the weight defaults to
#  1.0.
def getBalancedObjective1(si):
    lengthObj = ob.PathLengthOptimizationObjective(si)
    clearObj = ClearanceObjective(si)

    opt = ob.MultiOptimizationObjective(si)
    opt.addObjective(lengthObj, 5.0)
    opt.addObjective(clearObj, 1.0)

    return opt

## Create an optimization objective equivalent to the one returned by
#  getBalancedObjective1(), but use an alternate syntax.
#  THIS DOESN'T WORK YET. THE OPERATORS SOMEHOW AREN'T EXPORTED BY Py++.
# def getBalancedObjective2(si):
#     lengthObj = ob.PathLengthOptimizationObjective(si)
#     clearObj = ClearanceObjective(si)
#
#     return 5.0*lengthObj + clearObj


## Create an optimization objective for minimizing path length, and
#  specify a cost-to-go heuristic suitable for this optimal planning
#  problem.
def getPathLengthObjWithCostToGo(si):
    obj = ob.PathLengthOptimizationObjective(si)
    obj.setCostToGoHeuristic(ob.CostToGoHeuristic(ob.goalRegionCostToGo))
    return obj


# Keep these in alphabetical order and all lower case
def allocatePlanner(si, plannerType):
    if plannerType.lower() == "bfmtstar":
        return og.BFMT(si)
    elif plannerType.lower() == "bitstar":
        return og.BITstar(si)
    elif plannerType.lower() == "fmtstar":
        return og.FMT(si)
    elif plannerType.lower() == "informedrrtstar":
        return og.InformedRRTstar(si)
    elif plannerType.lower() == "prmstar":
        return og.PRMstar(si)
    elif plannerType.lower() == "rrtstar":
        return og.RRTstar(si)
    elif plannerType.lower() == "sorrtstar":
        return og.SORRTstar(si)
    elif plannerType.lower() == "sparstwo":
        return og.SPARStwo(si)
    else:
        ou.OMPL_ERROR("Planner-type is not implemented in allocation function.")


# Keep these in alphabetical order and all lower case
def allocateObjective(si, objectiveType):
    if objectiveType.lower() == "pathclearance":
        return getClearanceObjective(si)
    elif objectiveType.lower() == "pathlength":
        return getPathLengthObjective(si)
    elif objectiveType.lower() == "thresholdpathlength":
        return getThresholdPathLengthObj(si)
    elif objectiveType.lower() == "weightedlengthandclearancecombo":
        return getBalancedObjective1(si)
    else:
        ou.OMPL_ERROR("Optimization-objective is not implemented in allocation function.")



def plan(runTime, plannerType, objectiveType, interpolationnumber, min_distance, start_xyz, goal_xyz, max_runtime, optimizingTime, env_num):
    start_time = time.time()
    col_id, col_robot, collision_pairs = setup_collision_env(env_num)
    global validity_time, setup_time, solve_time
    timers = {"general": 0.0, "collision": 0.0, "validity": 0.0, "load_env": 0.0}
    
    start_setup_time = time.time()
    # 1. Setup des StateSpace
    space = ob.RealVectorStateSpace(7)
    bounds = ob.RealVectorBounds(7)
    for i in range(7):
        bounds.setLow(i, joint_limits_rad_min[i])
        bounds.setHigh(i, joint_limits_rad_max[i])
    space.setBounds(bounds)

    # 2. Inverse Kinematics für Start
    prev_angle =  [-0.45349761, -0.02002243,  2.21807648, -2.49418107,  0.48719819,  2.70876941,  2.82777177]
    full_prev_angle = franka_chain.active_to_full(prev_angle, np.zeros(len(franka_chain.links)))
    
    start_frame = np.eye(4)
    start_frame[:3, 3] = start_xyz
    start_frame[:3, :3] = R.from_euler('x', 180, degrees=True).as_matrix()
    joints_start = franka_chain.inverse_kinematics_frame(start_frame, initial_position=full_prev_angle)
    
    # 3. Inverse Kinematics für Goal
    target_frame = np.eye(4)
    target_frame[:3, 3] = goal_xyz
    target_frame[:3, :3] = R.from_euler('x', 180, degrees=True).as_matrix()
    joints_goal = franka_chain.inverse_kinematics_frame(target_frame, initial_position=full_prev_angle, orientation_mode="all")

    # Max. 10 Versuche, gültigen Startzustand zu finden
    for attempt in range(10):
        if calculate_state_validity(joints_start, col_robot, collision_pairs, col_id):
            break
        print(f"[Start-IK] Ungültiger Startzustand. Versuche neuen IK-Seed (Versuch {attempt+1}/10)")
        prev_angle = np.random.uniform(joint_limits_rad_min, joint_limits_rad_max).tolist()
        full_prev_angle = franka_chain.active_to_full(prev_angle, np.zeros(len(franka_chain.links)))
        joints_start = franka_chain.inverse_kinematics_frame(start_frame, initial_position=full_prev_angle)
    else:
        print("Fehler: Kein gültiger Startzustand nach 10 Versuchen.")
        return None, None, None, timers
    
    # Max. 10 Versuche, gültigen Zielzustand zu finden
    for attempt in range(10):
        if calculate_state_validity(joints_start, col_robot, collision_pairs, col_id):
            break
        print(f"[Ziel-IK] Ungültiger Zielzustand. Versuche neuen IK-Seed (Versuch {attempt+1}/10)")
        prev_angle = np.random.uniform(joint_limits_rad_min, joint_limits_rad_max).tolist()
        full_prev_angle = franka_chain.active_to_full(prev_angle, np.zeros(len(franka_chain.links)))
        joints_goal = franka_chain.inverse_kinematics_frame(target_frame, initial_position=full_prev_angle, orientation_mode="all")
    else:
        print("Fehler: Kein gültiger Zielzustand nach 10 Versuchen.")
        return None, None, None, timers


    start = ob.State(space)
    for i in range(7):
        start[i] = joints_start[i+1]
    goal = ob.State(space)
    
    for i in range(7):
        goal[i] = joints_goal[i+1]

    fks=franka_chain.forward_kinematics(joints_start)
    fkg=franka_chain.forward_kinematics(joints_goal)
    print(joints_start, joints_goal)
    print(fks, fkg)
        
    # Construct a space information instance for this state space
    si = ob.SpaceInformation(space)

    # Set the object used to check which states in the space are valid
    validityChecker = ValidityChecker(si, min_distance, timers, col_robot, collision_pairs, col_id)
    si.setStateValidityChecker(validityChecker)

    motion_validator = ob.DiscreteMotionValidator(si)
    si.setMotionValidator(motion_validator)
    si.setup()
    setup_time += time.time() - start_setup_time
    # Create a problem instance
    pdef = ob.ProblemDefinition(si)
    # Set the start and goal states
    pdef.setStartAndGoalStates(start, goal)

    # Create the optimization objective specified by our command-line argument.
    # This helper function is simply a switch statement.
    pdef.setOptimizationObjective(allocateObjective(si, objectiveType))

    # Construct the optimal planner specified by our command line argument.
    # This helper function is simply a switch statement.
    optimizingPlanner = allocatePlanner(si, plannerType)

    # Set the problem instance for our planner to solve
    optimizingPlanner.setProblemDefinition(pdef)
    optimizingPlanner.setup()
    
    
    # attempt to solve the planning problem in the given runtime
    start_solve_time = time.time()
    solved = optimizingPlanner.solve(runTime)
    solve_time += time.time() - start_solve_time
    
    
    while (str(solved) != "Exact solution"): 
        if time.time() - start_time >= max_runtime:
            print("Maximale runtime erreicht! Keine exakte Lösung gefunden.")
            break
        solved = optimizingPlanner.solve(runTime)
    
    if str(solved) == "Exact solution":
        timers["general"]+= time.time() - start_time
        print("Exakte lösung gefunden in: ", timers["general"], " Sekunden")
        first_path = pdef.getSolutionPath()
        first_path.interpolate(interpolationnumber)
        first_path_states = first_path.getStates()
        first_sol = [[s[i] for i in range(7)] for s in first_path_states]
        print(first_sol)
        np.save("first_sol.npy", first_sol)
        initial_cost = pdef.getSolutionPath().cost(pdef.getOptimizationObjective()).value()
        print('{0} found solution of path length {1:.4f} with an optimization ' \
            'objective value of {2:.4f}'.format( \
            optimizingPlanner.getName(), \
            pdef.getSolutionPath().length(), \
            initial_cost))

        #optimiting Path
        optimizingPlanner.solve(optimizingTime)
        path = pdef.getSolutionPath()
        path.interpolate(interpolationnumber)

        cost = pdef.getSolutionPath().cost(pdef.getOptimizationObjective()).value()
        print('{0} found solution of path length {1:.4f} with an optimization ' \
            'objective value of {2:.4f}'.format( \
            optimizingPlanner.getName(), \
            pdef.getSolutionPath().length(), \
            cost))
        
        path_states = path.getStates()
        opt_sol = [[s[i] for i in range(7)] for s in path_states]
    
        return opt_sol, initial_cost, cost, timers
    else:
        print("No solution found.")
        return (None, None, None, timers)

if __name__ == "__main__":
    runtime = 2.0
    plannerType = "informedrrt"
    objectiveType = "PathLength"
    interpolationnumber = 20
    min_distance = 0.01
    max_runtime = 120 
    optimizingTime = 20
    
    # start_xyz = [0.1, 0.1, 0.85]
    # goal_xyz = [0.65, 0.3, 0.4]
    # points, cost = plan(runtime, plannerType, objectiveType, interpolationnumber, min_distance, start_xyz, goal_xyz, max_runtime, optimizingTime)
    # np.save("ompl_angles.npy", points)
    # print(points)
    # print(j)
    # np.save("tested_joint_states.npy", tested_joint_states)
## @endcond

In [4]:
def run_planner(run_id, plannerType, objectiveType, start_xyz, goal_xyz, env_num,
                interpolationnumber=20, min_distance=0.01, max_runtime=100, optimizingTime=10):
    
    points, initial_cost, cost, timers = plan(
        runTime=2.0,
        plannerType=plannerType,
        objectiveType=objectiveType,
        interpolationnumber=interpolationnumber,
        min_distance=min_distance,
        start_xyz=start_xyz,
        goal_xyz=goal_xyz,
        max_runtime=max_runtime,
        optimizingTime=optimizingTime,
        env_num = env_num
    )
    
    return {
        "run_id": run_id,
        "planner": plannerType,
        "objective": objectiveType,
        "initial_cost": initial_cost,
        "cost": cost,
        "path": points,
        "success": points is not None,
        "timers": timers,
        "setup_time": setup_time,
        "solve_time": solve_time,
    }
#p = run_planner(1, "RRTstar", "PathLength", [0.1, 0.1, 0.85], [0.65, 0.3, 0.4])


In [ ]:
import os
import csv
results = []
repetitions = 10

planner = "rrtstar"
objective = "PathLength"
env_num = 3

if env_num == 1:
    # env 1 states
    start = [0.1, 0.1, 0.85]
    goal = [0, 0.5, 0.1]
    output_dir = "results_env1.2"
elif env_num == 2:
    # env 2 states
    start = [0.4, 0.4, 0.4]
    goal = [0, 0.5, 0.1]
    output_dir = "results_env2.3"
elif env_num == 3:
    # env 3 states
    start = [0.5, 0, 0.1]
    goal = [0, 0.5, 0.1]
    output_dir = "results_env3.3"
elif env_num == 4:
    # env 4 states
    start = [-0.1, 0.5, 0.2]
    goal = [0.69, 0, 0.05]
    output_dir = "results_env4.2"
    
os.makedirs(output_dir, exist_ok=True)

for i in range(repetitions):
    print(f"Run {i+1}/{repetitions} for {planner} + {objective}")
    result = run_planner(
        run_id=i,
        plannerType=planner,
        objectiveType=objective,
        start_xyz=start,
        goal_xyz=goal,
        env_num=env_num
    )
    results.append(result)

summary_path = os.path.join(output_dir, f"run_summary_{planner}.csv")
file_exists = os.path.isfile(summary_path)
    
with open(summary_path, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["run_id", "planner", "objective", "initial_cost", "cost", "path", "general_time", "validity_time", "collisionchecker_time", "env_time", "setup_time", "solve_time"])
    if not file_exists:
        writer.writeheader()
    for result in results:
        writer.writerow({
            "run_id": result["run_id"],
            "planner": result["planner"],
            "objective": result["objective"],
            "initial_cost": result["initial_cost"],
            "cost": result["cost"],
            "path": result["path"],
            "general_time": result["timers"]["general"],
            "validity_time": result["timers"]["validity"],
            "collisionchecker_time": result["timers"]["collision"],
            "env_time": result["timers"]["load_env"],
            "setup_time": result["setup_time"],
            "solve_time": result["solve_time"],
        })


Run 1/10 for rrtstar + PathLength
1
[ 0.         -0.78386157  0.44723351  0.79440907 -2.27044016 -0.64703223
  2.35505938 -0.7206836   0.          0.          0.          0.        ] [ 0.         -0.84253467 -0.36027342  2.46092396 -2.37981703  0.460114
  2.61844862  0.45968429  0.          0.          0.          0.        ]
[[ 8.98570116e-01  4.22009937e-01 -1.20330213e-01  4.99999971e-01]
 [ 4.08115820e-01 -9.04428459e-01 -1.24300603e-01 -1.17602226e-08]
 [-1.61286159e-01  6.25841437e-02 -9.84921317e-01  1.00000034e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] [[ 1.00000000e+00  1.84742200e-10 -4.61308700e-10 -1.92777610e-10]
 [ 1.84741750e-10 -1.00000000e+00 -2.47229998e-10  4.99999999e-01]
 [-4.61308641e-10  2.47229890e-10 -1.00000000e+00  1.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Invalid state: [0.1914299778974099, 1.0376772967337704, 0.5064807636540853, -1.0296635295891414, 0.2532605139493165, 1.089332282722090

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd

rrt = pd.read_csv("ompl_results/run_summary_RRTstar.csv")

fig, axs = plt.subplots(1, 2, figsize=(12, 5))

plt.suptitle("RRT*", fontsize=16)
# Erster Plot: time
axs[0].plot(rrt["time"], marker='o', linestyle='-')
axs[0].axhline(rrt["time"].mean(), linestyle='--', color="red", label="Durchschnitt")
axs[0].set_title("Laufzeit (Time)")
axs[0].set_xlabel("Versuch")
axs[0].set_ylabel("Zeit in Sekunden")
axs[0].set_ylim(0, 100)

# Zweiter Plot: cost
axs[1].plot(rrt["initial_cost"], marker='o', linestyle='-')
axs[1].axhline(rrt["initial_cost"].mean(), linestyle='--', color="red", label="Durchschnitt")
axs[1].set_title("Kosten (Cost)")
axs[1].set_xlabel("Versuch")
axs[1].set_ylabel("Pfadkosten")
axs[1].set_ylim(0, 10)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os

# Liste deiner Algorithmus-Dateien und Namen
algorithms = {
    "RRTstar": "ompl_results/run_summary_RRTstar.csv",
    "PRMstar": "ompl_results/run_summary_prmstar.csv",
    "Informedrrtstar": "ompl_results/run_summary_informedrrtstar.csv"
}

num_algos = len(algorithms)
fig, axs = plt.subplots(num_algos, 2, figsize=(12, 5 * num_algos))

# Falls nur ein Algorithmus, axs in 2D-Array konvertieren
if num_algos == 1:
    axs = [axs]

for idx, (name, path) in enumerate(algorithms.items()):
    df = pd.read_csv(path)

    # Plot Time
    axs[idx][0].plot(df["time"], marker='o', linestyle='-')
    axs[idx][0].axhline(df["time"].mean(), linestyle='--', color="red", label="Durchschnitt")
    axs[idx][0].set_title(f"{name}: Laufzeit")
    axs[idx][0].set_xlabel("Versuch")
    axs[idx][0].set_ylabel("Zeit in Sekunden")
    axs[idx][0].set_ylim(0, 100)
    axs[idx][0].legend()

    # Plot Cost
    axs[idx][1].plot(df["cost"], marker='o', linestyle='-')
    axs[idx][1].axhline(df["cost"].mean(), linestyle='--', color="red", label="Durchschnitt")
    axs[idx][1].set_title(f"{name}: Pfadkosten")
    axs[idx][1].set_xlabel("Versuch")
    axs[idx][1].set_ylabel("Pfadkosten")
    axs[idx][1].set_ylim(0, 10)
    axs[idx][1].legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])  # Platz für suptitle lassen
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


def plot_xyz(x, y, z, start_xyz, goal_xyz, tested_points= None):
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 3, 1, projection='3d')
    firx = fig.add_subplot(1, 3, 2, projection='3d')
    finx = fig.add_subplot(1, 3, 3, projection='3d')
    configure_3d_axes(ax)
    configure_3d_axes(firx)
    configure_3d_axes(finx)
    ax.set_title("Sampled Points")
    firx.set_title("First Trajectory")
    finx.set_title("Final Trajectory")

    start_xyz = np.array(start_xyz)
    goal_xyz = np.array(goal_xyz)

    first_traj= np.load("first_sol.npy")
    first_traj_full =[]
    first_traj_fk =[]
    for i in range(len(first_traj)):
        first_traj_full.append(franka_chain.active_to_full(first_traj[i], np.zeros(len(franka_chain.links))))
        first_traj_fk.append(franka_chain.forward_kinematics(first_traj_full[i]))

    first_sol = np.array([traj[0:3, 3] for traj in first_traj_fk])
    
    final_traj= np.load("ompl_angles.npy")
    full_traj =[]
    final_traj_fk =[]
    for i in range(len(final_traj)):
        full_traj.append(franka_chain.active_to_full(final_traj[i], np.zeros(len(franka_chain.links))))
        final_traj_fk.append(franka_chain.forward_kinematics(full_traj[i]))

    final_sol = np.array([traj[0:3, 3] for traj in final_traj_fk])


    
    # if tested_points is not None:
    #     ax.scatter(tested_points[0], tested_points[1], tested_points[2], c='gray', marker='.', alpha=0.3, label='Checked points')

    
    # Geplante Punkte + Start + Ziel
    ax.scatter(x, y, z, c='b', marker='o', label='Tested Points',zorder=1)
    ax.scatter(start_xyz[0], start_xyz[1], start_xyz[2], c='r', marker='o', label='Start')
    ax.scatter(goal_xyz[0], goal_xyz[1], goal_xyz[2], c='g', marker='o', label='Goal')

     # Erste gelöste Trajektorie + Start + Ziel
    firx.scatter(first_sol[:, 0], first_sol[:, 1], first_sol[:, 2], c='black', marker='o', label='First Trajectory', zorder=5)
    firx.scatter(start_xyz[0], start_xyz[1], start_xyz[2], c='r', marker='o', label='Start')
    firx.scatter(goal_xyz[0], goal_xyz[1], goal_xyz[2], c='g', marker='o', label='Goal')

    # Finale Trajektorie + Start + Ziel
    finx.scatter(final_sol[:, 0], final_sol[:, 1], final_sol[:, 2], c='black', marker='o', label='Final Trajectory', zorder=5)
    finx.scatter(start_xyz[0], start_xyz[1], start_xyz[2], c='r', marker='o', label='Start')
    finx.scatter(goal_xyz[0], goal_xyz[1], goal_xyz[2], c='g', marker='o', label='Goal')

    plt.show()
    print(len(x))

def configure_3d_axes(ax):
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-1, 1)
    ax.legend()
  
    
plot_xyz(all_x, all_y, all_z, [0.1, 0.1, 0.85], [0.65, 0.3, 0.4])

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Dictionary mit Namen und zugehörigen CSV-Dateien
algorithms = {
    "RRTstar": "results_env4.2run_summary_rrtstar.csv",
    "PRMstar": "results_env4.2run_summary_prmstar.csv",
}
colors = ["#377eb8", "tab:red"]

params = {
    'axes.labelsize': 16,      # Größere Achsenbeschriftungen
    'xtick.labelsize': 16,     # Große X-Achsen Werte
    'ytick.labelsize': 16,     # Große Y-Achsen Werte
    'axes.titlesize': 14,      # Großer Titel
    'legend.fontsize': 16,     # Große Legende
}
plt.rcParams.update(params)

# Funktion zur Datenbereinigung
def clean_data(df):
    # Ersetze fehlgeschlagene Runs (initial_cost = 0 oder NaN) mit Laufzeit = 100
    mask = (df["initial_cost"].isna()) | (df["initial_cost"] == 0)
    df.loc[mask, "general_time"] = 100
    df.loc[mask, "initial_cost"] = np.nan  # Markiere als fehlgeschlagen
    return df

# Laufzeitvergleich
fig_laufzeit = plt.figure(figsize=(6, 5))
ax1 = fig_laufzeit.add_subplot(1, 1, 1)

for i, (name, path) in enumerate(algorithms.items()):
    df = pd.read_csv(path)
    #df = clean_data(df)

    x = range(1, len(df) + 1)
    ax1.plot(x, df["general_time"],
             label=f"{name} (Avg: {df['general_time'].mean():.1f}s)",
             marker='o', linestyle='-', color=colors[i])
    ax1.axhline(df["general_time"].mean(), linestyle='--', color=colors[i], alpha=0.3)
    print("time",df["general_time"].mean())

ax1.set_xlabel("Versuchsdurchlauf", labelpad=10)
ax1.set_ylabel("Zeit [s]", labelpad=10)
ax1.set_ylim(0, 105)
ax1.grid(alpha=0.2)
ax1.legend()
fig_laufzeit.tight_layout()
fig_laufzeit.savefig("laufzeitvergleich3.svg", format='svg', dpi=300, bbox_inches='tight')

# Pfadkostenvergleich (nur erfolgreiche Runs)
fig_kosten = plt.figure(figsize=(6,5))
ax2 = fig_kosten.add_subplot(1, 1, 1)

for i, (name, path) in enumerate(algorithms.items()):
    df = pd.read_csv(path)
    df = clean_data(df)
    successful_runs = df.dropna()  # Filtere fehlgeschlagene Runs

    if not successful_runs.empty:
        x = range(1, len(successful_runs) + 1)
        ax2.plot(x, successful_runs["initial_cost"],
                 label=f"{name} (Avg: {successful_runs['initial_cost'].mean():.1f})",
                 marker='o', linestyle='-', color=colors[i])
        ax2.axhline(successful_runs["initial_cost"].mean(),
                   linestyle='--', color=colors[i], alpha=0.3)
        print("cost",successful_runs["initial_cost"].mean())
        print("std",successful_runs["initial_cost"].std())

ax2.set_title("Pfadkostenvergleich")
ax2.set_xlabel("Versuchsdurchlauf", labelpad=10)
ax2.set_ylabel("Pfadkosten", labelpad=10)
ax2.set_ylim(0, 30)
ax2.grid(alpha=0.2)
ax2.legend()
fig_kosten.tight_layout()
fig_kosten.savefig("pfadkostenvergleich3.svg", format='svg', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import ast


def plot_xyz(x, y, z, start_xyz, goal_xyz, tested_points= None):
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 3, 1, projection='3d')
    firx = fig.add_subplot(1, 3, 2, projection='3d')
    configure_3d_axes(ax)
    configure_3d_axes(firx)

    ax.set_title("Sampled Points")
    firx.set_title("First Trajectory")


    start_xyz = np.array(start_xyz)
    goal_xyz = np.array(goal_xyz)

    rrt = pd.read_csv("ompl_results_env4/run_summary_RRTstar.csv")
    rrt = rrt[rrt["path"].notna()]
    first_traj = [ast.literal_eval(traj_str) for traj_str in rrt["path"]]
    first_traj_full =[]
    first_traj_fk =[]
    for i in range(len(first_traj)):
        first_traj_full.append(franka_chain.active_to_full(first_traj[i], np.zeros(len(franka_chain.links))))
        first_traj_fk.append(franka_chain.forward_kinematics(first_traj_full[i]))

    first_sol = np.array([traj[0:3, 3] for traj in first_traj_fk])
    

    
    # if tested_points is not None:
    #     ax.scatter(tested_points[0], tested_points[1], tested_points[2], c='gray', marker='.', alpha=0.3, label='Checked points')

    
    # Geplante Punkte + Start + Ziel
    ax.scatter(x, y, z, c='b', marker='o', label='Tested Points',zorder=1)
    ax.scatter(start_xyz[0], start_xyz[1], start_xyz[2], c='r', marker='o', label='Start')
    ax.scatter(goal_xyz[0], goal_xyz[1], goal_xyz[2], c='g', marker='o', label='Goal')

     # Erste gelöste Trajektorie + Start + Ziel
    firx.scatter(first_sol[:, 0], first_sol[:, 1], first_sol[:, 2], c='black', marker='o', label='First Trajectory', zorder=5)
    firx.scatter(start_xyz[0], start_xyz[1], start_xyz[2], c='r', marker='o', label='Start')
    firx.scatter(goal_xyz[0], goal_xyz[1], goal_xyz[2], c='g', marker='o', label='Goal')


    plt.show()
    print(len(x))

def configure_3d_axes(ax):
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_zlim(-1, 1)
    ax.legend()
  
    
plot_xyz(all_x, all_y, all_z, [0.1, 0.1, 0.85], [0.65, 0.3, 0.4])

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Daten laden
rrt_df = pd.read_csv("run_summary_rrtstar.csv")
prm_df = pd.read_csv("run_summary_prmstar.csv")
irrt_df = pd.read_csv("run_summary_informedrrtstar.csv")

# Funktion zum Erstellen der Plots mit Durchschnittslinien
def plot_with_avg(df, title, color1, color2, filename):
    plt.figure(figsize=(6, 5))

    # Initiale Kosten
    initial_avg = df["initial_cost"].mean()
    plt.plot(df["initial_cost"], 'o-', color=color1,
             label=f'Initial (Avg: {initial_avg:.2f})', alpha=0.5, markersize=6)
    plt.axhline(initial_avg, color=color1, linestyle='--', alpha=0.7)

    # Optimierte Kosten
    optimized_avg = df["cost"].mean()
    plt.plot(df["cost"], 'o-', color=color2,
             label=f'Optimiert (Avg: {optimized_avg:.2f})', alpha=0.5, markersize=6)
    plt.axhline(optimized_avg, color=color2, linestyle='--', alpha=0.7)

    # Füllbereich
    plt.fill_between(np.arange(len(df)),
                    df["initial_cost"],
                    df["cost"],
                    color=color2, alpha=0.1)

    plt.xlabel("Durchlauf", fontsize=16)
    plt.ylabel("Pfadkosten", fontsize=16)
    plt.legend(fontsize=12, loc='upper right')
    plt.grid(True, alpha=0.2)
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

# Plots erstellen
plot_with_avg(rrt_df, "RRT*", 'tab:red', 'tab:green', "rrt_cost_comparison.png")
plot_with_avg(prm_df, "PRM*", 'tab:blue', 'tab:purple', "prm_cost_comparison.svg")
plot_with_avg(irrt_df, "Informed RRT*", 'tab:red', 'tab:green', "informed_rrt_cost_comparison.svg")

# Mittelwerte ausgeben
print(f"RRT* initial: {rrt_df['initial_cost'].mean():.2f}")
print(f"RRT* optimiert: {rrt_df['cost'].mean():.2f}")
print(f"PRM* initial: {prm_df['initial_cost'].mean():.2f}")
print(f"PRM* optimiert: {prm_df['cost'].mean():.2f}")
print(f"Informed RRT* initial: {irrt_df['initial_cost'].mean():.2f}")
print(f"Informed RRT* optimiert: {irrt_df['cost'].mean():.2f}")